### Bayesian Network Inference Library

We will be using the pomegranate library for Bayes Net inference

  * Installation instructions https://pomegranate.readthedocs.io/en/latest/install.html
  * Tutorial / documentation https://pomegranate.readthedocs.io/en/latest/BayesianNetwork.html
  
In the tutorial / documentation, ignore the parts about "initializing a Bayesian network based completely on data" and the sections on "Probability" "Prediction" and "Fitting" -- see the example below on how to determine the probability distribution on a node in the graph based on evidence.


In the cell below is the "alarm network" example from the class slides.

![Alarm](alarm.GIF)

In [6]:
from pomegranate import *

# Define probability distributions for the root nodes (Burglary & Earthquake)
burglary = DiscreteDistribution({'+b': 0.001, '-b': 0.999})
earthquake = DiscreteDistribution({'+e': 0.002, '-e': 0.998})

# Conditional probability distribution for the Alarm node, given Burglary and Earthquake
alarm = ConditionalProbabilityTable(
    [['+b', '+e', '+a', 0.95],
     ['+b', '+e', '-a', 0.05],
     ['+b', '-e', '+a', 0.94],
     ['+b', '-e', '-a', 0.06],
     ['-b', '+e', '+a', 0.29],
     ['-b', '+e', '-a', 0.71],
     ['-b', '-e', '+a', 0.001],
     ['-b', '-e', '-a', 0.999]],
   [burglary, earthquake]
)

# Conditional probability distribution for John Calls, given Alarm
john_calls = ConditionalProbabilityTable(
    [['+a', '+j', 0.9],
     ['+a', '-j', 0.1],
     ['-a', '+j', 0.05],
     ['-a', '-j', 0.95]],
    [alarm]
)

# Conditional probability distribution for Mary Calls, given Alarm
mary_calls = ConditionalProbabilityTable(
    [['+a', '+m', 0.7],
     ['+a', '-m', 0.3],
     ['-a', '+m', 0.01],
     ['-a', '-m', 0.99]],
    [alarm]
)

# Create states for each node
s_burglary = State(burglary, name = 'Burglary')
s_earthquake = State(earthquake, name = 'Earthquake')
s_alarm = State(alarm, name = 'Alarm')
s_john_calls = State(john_calls, name = 'John Calls')
s_mary_calls = State(mary_calls, name = 'Mary Calls')

# Initialize the Bayesian Network
model = BayesianNetwork("Alarm Network")

# Add states to the network
model.add_states(s_burglary, s_earthquake, s_alarm, s_john_calls, s_mary_calls)

# Add edges between the states to represent the conditional dependencies
model.add_edge(s_burglary, s_alarm)
model.add_edge(s_earthquake, s_alarm)
model.add_edge(s_alarm, s_john_calls)
model.add_edge(s_alarm, s_mary_calls)

# Bake model
model.bake()

In [7]:
# Compute and print the probability that there is an earthquake, based on no evidence
beliefs = model.predict_proba({})
earthquake_belief = beliefs[model.states.index(s_earthquake)]

print("The probability of an earthquake with no evidence is:")
print(earthquake_belief)

The probability of an earthquake with no evidence is:
{
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "+e" : 0.0020000000000004424,
            "-e" : 0.9979999999999996
        }
    ],
    "frozen" : false
}


In [8]:
# Based on no more evidence, what is the likelihood that the alarm goes off
# Simply compute and print the probability
alarm_belief = beliefs[model.states.index(s_alarm)]

print("The likelihood of the alarm going off based on no evidence is: ")
print(alarm_belief)

The likelihood of the alarm going off based on no evidence is: 
{
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "-a" : 0.997483557999999,
            "+a" : 0.0025164420000009344
        }
    ],
    "frozen" : false
}


In [26]:
##  Suppose you get a call from Mary but not John, what is the probability
##  that there was an earthquarke.  This cell should compute and print that
##  probability.
beliefs = model.predict_proba({'Mary Calls': '+m', 'John Calls': '-j'})
earthquake_belief = beliefs[model.states.index(s_earthquake)]

print("The probability of an earthquake given a call from Mary but not John is:")
print(earthquake_belief)

The probability of an earthquake given a call from Mary but not John is:
{
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "+e" : 0.0056121515205590665,
            "-e" : 0.994387848479441
        }
    ],
    "frozen" : false
}


In [14]:
# Compute and print the probability that the alarm went off and there was a burglary,
# given that both John and Mary called
beliefs = model.predict_proba({'John Calls': '+j', 'Mary Calls': '+m'})
burglary_belief = beliefs[model.states.index(s_burglary)]#.parameters[0]['+b']

print(f"Probability of Burglary given that both John and Mary called (and thus the alarm went off):")
print(burglary_belief)

Probability of Burglary given that both John and Mary called (and thus the alarm went off):
{
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "+b" : 0.2841718353644582,
            "-b" : 0.7158281646355419
        }
    ],
    "frozen" : false
}


In [24]:
## Compute and print the probability that there was an earthquake OR a burglary given no evidence
beliefs = model.predict_proba({})
earthquake_belief = beliefs[model.states.index(s_earthquake)].parameters[0]['+e']
burglary_belief = beliefs[model.states.index(s_burglary)].parameters[0]['+b']

# Calculate the probability of both an earthquake and a burglary happening
prob_both = earthquake_belief * burglary_belief

# Calculate the probability of either an earthquake or a burglary happening
prob_e_or_b = earthquake_belief + burglary_belief - prob_both

print("Probability of either an Earthquake or a Burglary:", prob_e_or_b)

Probability of either an Earthquake or a Burglary: 0.0029980000000008845


In [27]:
## How much does your belief that there was an earthquake change on the basis of getting
## a call from John but not Mary.   Compute that change and print it as a percentage.
# Calculate initial belief in an earthquake with no evidence
initial_beliefs = model.predict_proba({})
initial_earthquake_probability = initial_beliefs[model.states.index(s_earthquake)].parameters[0]['+e']

# Calculate updated belief in an earthquake given evidence: a call from John but not from Mary
updated_beliefs = model.predict_proba({'JohnCalls': '+j', 'MaryCalls': '-m'})
updated_earthquake_probability = updated_beliefs[model.states.index(s_earthquake)].parameters[0]['+e']

# Calculate the change in belief
change_in_belief = updated_earthquake_probability - initial_earthquake_probability

# Express the change as a percentage of the initial belief
change_in_belief_percentage = (change_in_belief / initial_earthquake_probability) * 100

print("Initial probability of an Earthquake:", initial_earthquake_probability)
print("Updated probability of an Earthquake with evidence:", updated_earthquake_probability)
print("Change in belief of an Earthquake:", change_in_belief_percentage, "%")

Initial probability of an Earthquake: 0.0020000000000004424
Updated probability of an Earthquake with evidence: 0.0020000000000004424
Change in belief of an Earthquake: 0.0 %
